In [91]:
import numpy as np
from scipy.stats import norm
import tensorflow as tf
import keras
from keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from FFNDLResearchModels import *

## The Underlying Instruments

In [94]:
class ZeroCouponBond:
    def __init__(self, maturity: float, riskFreeRate: float):
        self.riskFreeRate = riskFreeRate
        self.maturity = maturity

    def price(self):
        value = np.exp(-self.riskFreeRate * self.maturity)
        return value

In [95]:
class AnalyticalVanillaEuropeanOption:
    def __init__(self, maturity: float, initialStockPrice: float, strikePrice: float, riskFreeRate: float, impliedVolatility: float, option_type: str):
        self.maturity = maturity
        self.initialStockPrice = initialStockPrice
        self.strikePrice = strikePrice
        self.riskFreeRate = riskFreeRate
        self.impliedVolatility = impliedVolatility
        self.option_type = option_type.lower()

    def __compute_d1_d2(self):
        d1 = (np.log(self.initialStockPrice / self.strikePrice) + 
              (self.riskFreeRate + 0.5 * self.impliedVolatility**2) * 
              self.maturity) / (self.impliedVolatility * np.sqrt(self.maturity))
        d2 = d1 - self.impliedVolatility * np.sqrt(self.maturity)
        return d1, d2
    
    def price(self):
        d1, d2 = self.__compute_d1_d2()
        if self.option_type == 'call':
            price = (self.initialStockPrice * norm.cdf(d1) - 
                     self.strikePrice * np.exp(-self.riskFreeRate * self.maturity) * 
                     norm.cdf(d2))
        else:  # Put option
            price = (self.strikePrice * np.exp(-self.riskFreeRate * self.maturity) * 
                     norm.cdf(-d2) - self.initialStockPrice * norm.cdf(-d1))
        return price
    
    def delta(self):
        d1, _ = self.__compute_d1_d2()
        if self.option_type == 'call':
            delta = norm.cdf(d1)
        else:  # Put option
            delta = -norm.cdf(-d1)
        return delta

In [96]:
class MonteCarloEuropeanOption:
    default_steps = 500
    default_simulations = 100000

    def __init__(self, maturity: float, initialStockPrice: float, strikePrice: float, riskFreeRate: float, impliedVolatility: float, option_type: str):
        self.maturity = maturity
        self.initialStockPrice = initialStockPrice
        self.strikePrice = strikePrice
        self.riskFreeRate = riskFreeRate
        self.impliedVolatility = impliedVolatility
        self.option_type = option_type.lower()
        self.steps = MonteCarloEuropeanOption.default_steps
        self.simulations = MonteCarloEuropeanOption.default_simulations

    def simulate_stock_price(self):
        dt = self.maturity / self.steps
        half_simulations = self.simulations // 2
        price_paths = np.zeros((self.steps, self.simulations))
        price_paths[0, :] = self.initialStockPrice
        for t in range(1, self.steps):
            Z = np.random.standard_normal(half_simulations)
            antithetic_Z = -Z
            Z = np.concatenate((Z, antithetic_Z))  # Combine normal and antithetic variates
            price_paths[t] = price_paths[t - 1] * np.exp((self.riskFreeRate - 0.5 * self.impliedVolatility**2) * dt + self.impliedVolatility * np.sqrt(dt) * Z)
        return price_paths

    def price(self):
        terminal_prices = self.simulate_stock_price()[-1]
        payoffs = np.maximum(terminal_prices - self.strikePrice, 0) if self.option_type == 'call' else np.maximum(self.strikePrice - terminal_prices, 0)
        discounted_payoff = np.exp(-self.riskFreeRate * self.maturity) * np.mean(payoffs)
        return discounted_payoff

    def delta(self):
        dt = self.maturity / self.steps
        price_paths = self.simulate_stock_price()
        terminal_prices = price_paths[-1]
        if self.option_type == 'call':
            in_the_money = terminal_prices > self.strikePrice
            payoffs = np.exp(-self.riskFreeRate * self.maturity) * in_the_money * (price_paths[-1] / self.initialStockPrice)
        else:
            in_the_money = terminal_prices < self.strikePrice
            payoffs = -np.exp(-self.riskFreeRate * self.maturity) * in_the_money * (price_paths[-1] / self.initialStockPrice)
        return np.mean(payoffs)

In [1]:
class AnalyticalVanillaEuropeanOption:
    def __init__(self, maturity: float, initialStockPrice: float, strikePrice: float, riskFreeRate: float, impliedVolatility: float, option_type: str):
        self.maturity = maturity
        self.initialStockPrice = initialStockPrice
        self.strikePrice = strikePrice
        self.riskFreeRate = riskFreeRate
        self.impliedVolatility = impliedVolatility
        self.option_type = option_type.lower()

    def __compute_d1_d2(self):
        d1 = (np.log(self.initialStockPrice / self.strikePrice) + 
              (self.riskFreeRate + 0.5 * self.impliedVolatility**2) * 
              self.maturity) / (self.impliedVolatility * np.sqrt(self.maturity))
        d2 = d1 - self.impliedVolatility * np.sqrt(self.maturity)
        return d1, d2
    
    def price(self):
        d1, d2 = self.__compute_d1_d2()
        if self.option_type == 'call':
            price = (self.initialStockPrice * norm.cdf(d1) - 
                     self.strikePrice * np.exp(-self.riskFreeRate * self.maturity) * 
                     norm.cdf(d2))
        else:  # Put option
            price = (self.strikePrice * np.exp(-self.riskFreeRate * self.maturity) * 
                     norm.cdf(-d2) - self.initialStockPrice * norm.cdf(-d1))
        return price
    
    def delta(self):
        d1, _ = self.__compute_d1_d2()
        if self.option_type == 'call':
            delta = norm.cdf(d1)
        else:  # Put option
            delta = -norm.cdf(-d1)
        return delta

In [97]:
class AnalyticalChooserOption:
    def __init__(self, time_to_choose: float, maturity: float, 
                 initialStockPrice: float, strikePrice: float, riskFreeRate: float, impliedVolatility: float):
        self.time_to_choose = time_to_choose
        self.maturity = maturity
        self.initialStockPrice = initialStockPrice
        self.strikePrice = strikePrice
        self.riskFreeRate = riskFreeRate
        self.impliedVolatility = impliedVolatility

    def __d_plus(self):
        T = self.maturity
        return ((np.log(self.initialStockPrice / self.strikePrice) + 
                (self.riskFreeRate + (self.impliedVolatility**2) / 2) * T) / 
                (self.impliedVolatility * np.sqrt(T)))

    def __d_minus(self, d_plus):
        T = self.maturity
        return d_plus - self.impliedVolatility * np.sqrt(T)

    def __d1(self):
        t = self.time_to_choose
        T = self.maturity
        return ((np.log(self.initialStockPrice / self.strikePrice) + 
                 self.riskFreeRate * T + (self.impliedVolatility**2 / 2) * t) / 
                (self.impliedVolatility * np.sqrt(t)))

    def __d2(self, d1):
        return d1 - self.impliedVolatility * np.sqrt(self.time_to_choose)

    def price(self):
        d_plus = self.__d_plus()
        d_minus = self.__d_minus(d_plus)
        d1 = self.__d1()
        d2 = self.__d2(d1)

        value = (self.initialStockPrice * (norm.cdf(d_plus) - norm.cdf(-d1)) +
                 self.strikePrice * np.exp(-self.riskFreeRate * self.maturity) * 
                 (norm.cdf(-d2) - norm.cdf(d_minus)))
        return value
    
    def delta(self):
        d_plus = self.__d_plus()
        d1 = self.__d1()
        return norm.cdf(d_plus) - norm.cdf(-d1)

In [98]:
class MonteCarloChooserOption:
    def __init__(self, time_to_choose: float, maturity: float, 
                 initialStockPrice: float, strikePrice: float, riskFreeRate: float, 
                 impliedVolatility: float, simulations: int = 1000000, steps: int = 500):
        self.time_to_choose = time_to_choose
        self.maturity = maturity
        self.initialStockPrice = initialStockPrice
        self.strikePrice = strikePrice
        self.riskFreeRate = riskFreeRate
        self.impliedVolatility = impliedVolatility
        self.simulations = simulations
        self.steps = steps

    def simulate_stock_price(self):
        dt = self.time_to_choose / self.steps
        price_paths = np.zeros((self.steps, self.simulations))
        price_paths[0, :] = self.initialStockPrice

        for t in range(1, self.steps):
            Z = np.random.standard_normal(self.simulations)
            price_paths[t] = price_paths[t - 1] * np.exp((self.riskFreeRate - 0.5 * self.impliedVolatility**2) * dt + self.impliedVolatility * np.sqrt(dt) * Z)
        
        return price_paths[-1]

    def calculate_payoff(self):
        final_prices = self.simulate_stock_price()
        # Decision for call or put at the choice time
        call_payoff = np.maximum(final_prices - self.strikePrice, 0)
        put_payoff = np.maximum(self.strikePrice - final_prices, 0)
        # Choose the higher payoff for each path
        payoffs = np.maximum(call_payoff, put_payoff)
        return payoffs

    def price(self):
        payoffs = self.calculate_payoff()
        # Discount the average payoff back to the present value
        discounted_payoff = np.exp(-self.riskFreeRate * self.maturity) * np.mean(payoffs)
        return discounted_payoff

In [134]:
# Create an instance of the class
chooser_option = AnalyticalChooserOption(
    time_to_choose=0.5, 
    maturity=1, 
    initialStockPrice=100, 
    strikePrice=110, 
    riskFreeRate=0.05, 
    impliedVolatility=0.20
)

# Calculate the price of the chooser option
option_price = chooser_option.price()

# Calculate the delta of the chooser option
option_delta = chooser_option.delta()

print(f"The price of the chooser option is: {option_price}")
print(f"The delta of the chooser option is: {option_delta}")

The price of the chooser option is: 14.418531412026764
The delta of the chooser option is: -0.14893491764979422


In [99]:
class AnalyticalCashOrNothingOption:
    def __init__(self, maturity: float, initialStockPrice: float, strikePrice: float, riskFreeRate: float, impliedVolatility: float, payout: float, option_type: str):
        self.maturity = maturity
        self.initialStockPrice = initialStockPrice
        self.strikePrice = strikePrice
        self.riskFreeRate = riskFreeRate
        self.impliedVolatility = impliedVolatility
        self.payout = payout
        self.kappa = 1 if option_type.lower() == 'call' else -1

    def __d_minus(self):
        return ((np.log(self.initialStockPrice / self.strikePrice) + 
                (self.riskFreeRate - (self.impliedVolatility**2) / 2) * self.maturity) / 
                (self.impliedVolatility * np.sqrt(self.maturity)))

    def price(self):
        d_minus = self.__d_minus()
        value = self.payout * np.exp(-self.riskFreeRate * self.maturity) * norm.cdf(self.kappa * d_minus)
        return value
    
    def delta(self):
        d_minus = self.__d_minus()
        phi_d_minus = norm.pdf(d_minus)  # Standard normal probability density function at d_minus
        delta = self.kappa * self.payout * np.exp(-self.riskFreeRate * self.maturity) * \
                phi_d_minus / (self.impliedVolatility * self.initialStockPrice * np.sqrt(self.maturity))
        return delta

In [100]:
class MonteCarloCashOrNothingOption:
    default_steps = 500
    default_simulations = 100000

    def __init__(self, maturity: float, initialStockPrice: float, strikePrice: float, riskFreeRate: float, impliedVolatility: float, payout: float, option_type: str):
        self.maturity = maturity
        self.initialStockPrice = initialStockPrice
        self.strikePrice = strikePrice
        self.riskFreeRate = riskFreeRate
        self.impliedVolatility = impliedVolatility
        self.payout = payout
        self.option_type = option_type.lower()
        self.steps = MonteCarloCashOrNothingOption.default_steps
        self.simulations = MonteCarloCashOrNothingOption.default_simulations

    def simulate_stock_price(self):
        dt = self.maturity / self.steps
        half_simulations = self.simulations // 2
        price_paths = np.zeros((self.steps, self.simulations))
        price_paths[0, :] = self.initialStockPrice
        for t in range(1, self.steps):
            Z = np.random.standard_normal(half_simulations)
            antithetic_Z = -Z
            Z = np.concatenate((Z, antithetic_Z))  # Combine normal and antithetic variates
            price_paths[t] = price_paths[t - 1] * np.exp((self.riskFreeRate - 0.5 * self.impliedVolatility**2) * dt + self.impliedVolatility * np.sqrt(dt) * Z)
        return price_paths

    def price(self):
        terminal_prices = self.simulate_stock_price()[-1]
        if self.option_type == 'call':
            payoffs = np.where(terminal_prices > self.strikePrice, self.payout, 0)
        else:
            payoffs = np.where(terminal_prices < self.strikePrice, self.payout, 0)
        discounted_payoff = np.exp(-self.riskFreeRate * self.maturity) * np.mean(payoffs)
        return discounted_payoff
    
    
    def delta(self):
        epsilon = 0.01  # Small change in the initial stock price

        # Bump the stock price up and down
        bumped_up_price = self.initialStockPrice * (1 + epsilon)
        bumped_down_price = self.initialStockPrice * (1 - epsilon)

        # Save original stock price and set to bumped up price
        original_stock_price = self.initialStockPrice
        self.initialStockPrice = bumped_up_price
        terminal_prices_up = self.simulate_stock_price()[-1]

        # Reset to original and set to bumped down price
        self.initialStockPrice = bumped_down_price
        terminal_prices_down = self.simulate_stock_price()[-1]

        # Reset to original price
        self.initialStockPrice = original_stock_price

        if self.option_type == 'call':
            payoffs_up = np.where(terminal_prices_up > self.strikePrice, self.payout, 0)
            payoffs_down = np.where(terminal_prices_down > self.strikePrice, self.payout, 0)
        else:  # put option
            payoffs_up = np.where(terminal_prices_up < self.strikePrice, self.payout, 0)
            payoffs_down = np.where(terminal_prices_down < self.strikePrice, self.payout, 0)

        delta_estimate = (np.mean(payoffs_up) - np.mean(payoffs_down)) / (2 * epsilon * self.initialStockPrice)
        return np.exp(-self.riskFreeRate * self.maturity) * delta_estimate

## Data Generation

In [101]:
class GenerateSyntheticData:
    
    def __init__(self, nsamples: int):
        self.nsamples = nsamples

    def data(self):
        # Generating random data
        stockPrice = np.random.uniform(50, 500, self.nsamples)  # Spot price
        strikePrice = np.random.uniform(40, 600, self.nsamples)  # Strike price
        maturity = np.random.uniform(1/252, 5, self.nsamples)  # Time to maturity in years
        time_to_choose = np.random.uniform(1/252, maturity, self.nsamples) # generate time to choose for the chooser option
        riskFreeRate = np.random.uniform(0.01, 0.08, self.nsamples)  # Risk-free interest rate
        impliedVolatility = np.random.uniform(0.1, 0.5, self.nsamples)  # Volatility

        # Combining the data into a single array for scaling
        data = np.array([maturity, stockPrice, strikePrice, riskFreeRate, impliedVolatility])

        return data

# Parameters
nsamples = 20000
generator = GenerateSyntheticData(nsamples)
dataset = generator.data()

nsamplesTest = 5
gen = generator = GenerateSyntheticData(nsamplesTest)
dataTest = generator.data()

## The Study of the Three Structures

In [102]:
zcbPrice = ZeroCouponBond(dataset[0], dataset[3]).price() # the zero coupon bond
analyticalCallPrice = AnalyticalVanillaEuropeanOption(dataset[0], dataset[1], dataset[2], dataset[3], dataset[4], 'call').price()
analyticalCallDelta = AnalyticalVanillaEuropeanOption(dataset[0], dataset[1], dataset[2], dataset[3], dataset[4], 'call').delta()
vanillaCallStructure = zcbPrice + analyticalCallPrice

#We need time to chooser to be less than maturity
#time_to_choose = np.random.uniform(0.001, 3, nsamples)  # Time to maturity in years
#analyticalChooserPrice = AnalyticalChooserOption(dataset[0], dataset[1], dataset[2], dataset[3], dataset[4], 'call').price()
#analyticalChooserDelta = AnalyticalVanillaEuropeanOption(dataset[0], dataset[1], dataset[2], dataset[3], dataset[4], 'call').delta()

In [103]:
analyticalCallPrice, zcbPrice, vanillaCallStructure

(array([1.25826728e+02, 4.05174029e-02, 4.62565623e+01, ...,
        1.06970387e-61, 1.04828965e+02, 1.57353059e+00]),
 array([0.90631844, 0.97406133, 0.8923671 , ..., 0.99190327, 0.88295345,
        0.86323048]),
 array([126.73304653,   1.01457873,  47.14892941, ...,   0.99190327,
        105.71191821,   2.43676107]))

## Feedforward Neural Network model

In [104]:
# Define the neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(5,)),  # 5 inputs: maturity, stockPrice, strikePrice, riskFreeRate, volatility
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for option price
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

#_, accuracy = model.evaluate(dataset, vanillaCallStructure)
#print('Accuracy: %.2f' % (accuracy*100))

# Train the model
model.fit(dataset.T, vanillaCallStructure, epochs=100, verbose=0) #transpose the dataset

### Testing and Validation

In [105]:
vanillaCallStructureTest = ZeroCouponBond(dataTest[0], dataTest[3]).price()+AnalyticalVanillaEuropeanOption(dataTest[0], dataTest[1], dataTest[2], dataTest[3], dataTest[4], 'call').price()

In [106]:
vanillaCallStructureTest, model.predict(dataTest).T

1/1 [==============================] - 0s 53ms/step


(array([  0.96560195, 193.99361821,   0.96566891,   0.9848186 ,
        126.89561564]),
 array([[2.6821225e+02, 4.2008836e+04, 1.6396144e+05, 3.9305997e+00,
         2.2502499e+01]], dtype=float32))

In [107]:
# Generate test data
#X_test, y_test = generate_data(nsamples)  # Number of test samples

# Evaluate the model
#model.evaluate(X_test, y_test)

In [108]:
#predicted_prices = model.predict(X_test).flatten()